In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from xgboost import XGBClassifier


import warnings
warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

In [ ]:
RANDOM_STATE = 42
results = pd.DataFrame(columns = ['model', 'task', 'accuracy'])

https://archive.ics.uci.edu/ml/datasets/wine+quality

In [ ]:
data = pd.read_csv('winequality-red.csv', sep = ';')
data.head()

1.Разделите выборку на обучающую и тестовую в отношении 70%/30%, предварительно выделив целевую переменную (колонка 'quality').

In [ ]:

# Выделяем целевую переменную
X = data.drop(columns=['quality'])  # признаки
y = data['quality']  # целевая переменная

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Вывод размеров полученных выборок
print("Размер обучающей выборки:", X_train.shape, y_train.shape)
print("Размер тестовой выборки:", X_test.shape, y_test.shape)


### Ваш код ###

2.Оцените качество на тестовой выборке по метрике accuracy для классификаторов:

DecisionTreeClassifier

BaggingClassifier со 100 деревьями

RandomForestClassifier со 100 деревьями

Сравните результаты и напишите какой вывод можно сделать.

In [ ]:

# Инициализация классификаторов
dt_classifier = DecisionTreeClassifier(random_state=42)
bagging_classifier = BaggingClassifier(n_estimators=100, random_state=42)
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Обучение классификаторов на обучающей выборке
dt_classifier.fit(X_train, y_train)
bagging_classifier.fit(X_train, y_train)
random_forest_classifier.fit(X_train, y_train)

# Предсказание на тестовой выборке
dt_pred = dt_classifier.predict(X_test)
bagging_pred = bagging_classifier.predict(X_test)
rf_pred = random_forest_classifier.predict(X_test)

# Оценка качества на тестовой выборке
acc_DT = accuracy_score(y_test, dt_pred)
acc_Bagging = accuracy_score(y_test, bagging_pred)
acc_RF = accuracy_score(y_test, rf_pred)


### Ваш код ###

# acc_DT = 0
# acc_Bagging = 0
# acc_RF = 0
results.loc[0] = ['DecisionTreeClassifier', 'task2', acc_DT]
results.loc[1] = ['BaggingClassifier ', 'task2', acc_Bagging]
results.loc[2] = ['RandomForestClassifier', 'task2', acc_RF]
results

3.Посчитайте качество на тестовой выборке по метрике accuracy для классификатора RandomForestClassifier, используя значения деревьев:
    
10, 50, 100, 200, далее с шагом 200 до 5000 деревьев.
Постройте график зависимости качества от числа деревьев.



In [ ]:


# Задаем значения числа деревьев
num_estimators = list(range(10, 5001, 200))

# Инициализация списка для сохранения значений accuracy
accuracy_scores = []

# Обучение и оценка качества для каждого значения числа деревьев
for n in num_estimators:
    rf_classifier = RandomForestClassifier(n_estimators=n, random_state=42)
    rf_classifier.fit(X_train, y_train)
    rf_pred = rf_classifier.predict(X_test)
    accuracy = accuracy_score(y_test, rf_pred)
    accuracy_scores.append(accuracy)

# Построение графика
plt.figure(figsize=(10, 6))
plt.plot(num_estimators, accuracy_scores, marker='o', linestyle='-')
plt.title('Зависимость качества RandomForestClassifier от числа деревьев')
plt.xlabel('Число деревьев')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

### Ваш код ###

4.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек sklearn и xgboost. Сравните значение метрики accuracy по cross_val_score

In [ ]:

y_train = np.where(y_train > 5, y_train - 6, y_train)

y_test = np.where(y_test > 5, y_test - 6, y_test)



In [ ]:
# Инициализация классификаторов
gbc_classifier = GradientBoostingClassifier(random_state=42)
xgb_classifier = XGBClassifier(random_state=42)

# Оценка качества с помощью кросс-валидации для GradientBoostingClassifier
acc_sklearn = np.mean(cross_val_score(gbc_classifier, X_train, y_train, cv=5, scoring='accuracy'))

# Оценка качества с помощью кросс-валидации для XGBClassifier
acc_xgboost = np.mean(cross_val_score(xgb_classifier, X_train, y_train, cv=5, scoring='accuracy'))


### Ваш код ###

# acc_sklearn = 0
# acc_xgboost = 0
results.loc[3] = ['GradientBoostingClassifier', 'task4', acc_sklearn]
results.loc[4] = ['XGBClassifier', 'task4', acc_xgboost]
results

5.Подберите оптимальные параметры этих алгоритмов с помощью GridSearchCV(cv=3).
Параметры для оптимизации:

скорость обучения

количество деревьев

глубина деревьев

Сравните значение метрики accuracy. Выведите лучшие параметры алгоритмов.

In [ ]:


# Параметры для GradientBoostingClassifier
gbc_param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# Параметры для XGBClassifier
xgb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# GridSearchCV для GradientBoostingClassifier
gbc_grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42), gbc_param_grid, cv=3, scoring='accuracy')
gbc_grid_search.fit(X_train, y_train)

# GridSearchCV для XGBClassifier
xgb_grid_search = GridSearchCV(XGBClassifier(random_state=42), xgb_param_grid, cv=3, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train)


### Ваш код ###

best_params_sklearn = gbc_grid_search.best_params_
print('Параметры лучшей модели sklearn:\n', best_params_sklearn)
best_params_xgboost = xgb_grid_search.best_params_
print('Параметры лучшей модели xgboost:\n', best_params_xgboost)
acc_sklearn_cv = xgb_grid_search.best_score_
acc_xgboost_cv = gbc_grid_search.best_score_
results.loc[5] = ['GradientBoostingClassifier_CV', 'task5', acc_sklearn_cv]
results.loc[6] = ['XGBClassifier_CV', 'task5', acc_xgboost_cv]
results

6.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек lightgbm и catboost. Сравните значение метрики accuracy по cross_val_score по всем четырем реализациям.

In [ ]:


# Инициализация классификаторов
lgbm_classifier = LGBMClassifier(random_state=42)
catboost_classifier = CatBoostClassifier(random_state=42, silent=True)

# Оценка качества с помощью кросс-валидации для GradientBoostingClassifier
acc_lightgbm = cross_val_score(lgbm_classifier, X_train, y_train, cv=5, scoring='accuracy')

# Оценка качества с помощью кросс-валидации для XGBClassifier
acc_catboost = cross_val_score(catboost_classifier, X_train, y_train, cv=5, scoring='accuracy')

print("Среднее значение accuracy для LightGBM:", np.mean(acc_lightgbm))
print("Среднее значение accuracy для CatBoost:", np.mean(acc_lightgbm))


### Ваш код ###

results.loc[7] = ['LGBMClassifier', 'task6', acc_lightgbm]
results.loc[8] = ['CatBoostClassifier', 'task6', acc_catboost]
results

7.Подберите оптимальные параметры для алгоритмов градиентного бустинга из библиотек lightgbm и catboost с теми же условиями.
Сравните значение метрики accuracy. Выведите лучшие параметры алгоритмов.

In [ ]:


### Ваш код ###


# Параметры для LightGBM
lgbm_param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# Параметры для CatBoost
catboost_param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# GridSearchCV для LightGBM
lgbm_grid_search = GridSearchCV(LGBMClassifier(random_state=42), lgbm_param_grid, cv=3, scoring='accuracy')
lgbm_grid_search.fit(X_train, y_train)

# GridSearchCV для CatBoost
catboost_grid_search = GridSearchCV(CatBoostClassifier(random_state=42, silent=True), catboost_param_grid, cv=3, scoring='accuracy')
catboost_grid_search.fit(X_train, y_train)


best_params_lightgbm = lgbm_grid_search.best_params_
print('Параметры лучшей модели lightgbm:\n', best_params_lightgbm)
best_params_catboost = catboost_grid_search.best_params_
print('Параметры лучшей модели catboost:\n', best_params_catboost)
acc_lightgbm_cv = lgbm_grid_search.best_score_
acc_catboost_cv = catboost_grid_search.best_score_
results.loc[9] = ['LGBMClassifier_CV', 'task7', acc_lightgbm_cv]
results.loc[10] = ['CatBoostClassifier_CV', 'task7', acc_catboost_cv]

results

8.Подберите оптимальные параметры алгоритма из библиотеки xgbost с помощью [optuna](https://github.com/optuna/optuna) . Параметры для оптимизации:

скорость обучения

количество деревьев

глубина деревьев

Сравните результат с поиском по сетке из sklearn. Выведите лучшие параметры алгоритма, найденные даным способом.

In [ ]:


### Ваш код ###


def test(trial):
    params = {
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.1, 0.5]),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'max_depth': trial.suggest_int('max_depth', 3, 7)
    }
    
    xgb_classifier = XGBClassifier(random_state=42, **params)
    score = cross_val_score(xgb_classifier, X_train, y_train, cv=3, scoring='accuracy').mean()
    
    return score

study = optuna.create_study(direction='maximize')
study.optimize(test, n_trials=100)

best_params_optuna = study.best_params
best_score_optuna = study.best_value

print("Лучшие параметры XGBoost, найденные с помощью Optuna:", best_params_optuna)
print("Лучшее значение accuracy, найденное с помощью Optuna:", best_score_optuna)

# Сравнение с поиском по сетке из scikit-learn
xgb_param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

xgb_grid_search = GridSearchCV(XGBClassifier(random_state=42), xgb_param_grid, cv=3, scoring='accuracy')
xgb_grid_search.fit(X_train, y_train)

best_params_xgboost_optuna = xgb_grid_search.best_params_
acc_xgboost_optuna = xgb_grid_search.best_score_



print('Параметры лучшей модели xgboost:\n', best_params_xgboost_optuna)
results.loc[11] = ['XGBClassifier_optuna', 'task8', acc_xgboost_optuna]


In [ ]:
results

9.Выведите качество по метрике accuracy стэкинга (StackingClassifier) 4-х алгоритмов с базовыми параметрами градиентного бустинга.

In [ ]:

### Ваш код ###


# Инициализация базовых моделей
base_models = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('random_forest', RandomForestClassifier(random_state=42)),
    ('gradient_boosting', GradientBoostingClassifier(random_state=42)),
    ('ada_boost', AdaBoostClassifier(random_state=42))
]

# Инициализация мета-классификатора
meta_classifier = GradientBoostingClassifier(random_state=42)

# Инициализация StackingClassifier
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=meta_classifier)

# Оценка качества с помощью кросс-валидации для StackingClassifier
stacking_accuracy_scores = cross_val_score(stacking_classifier, X_train, y_train, cv=5, scoring='accuracy')


acc_stacking_default = stacking_accuracy_scores.mean()
results.loc[12] = ['Stacking', 'task10', acc_stacking_default]

results

10.Выведите качество по метрике accuracy стэкинга 4-х алгоритмов с оптимальными параметрами градиентного бустинга. Сравните результаты с предыдущим шагом и напишите какой вывод можно из этого сделать.

In [ ]:
results